In [0]:


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)
import boto3
from datetime import datetime
import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_{}
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
"""
    print datetime.today()
    db_df = []
    for device in [0,1,2]:
        db_device_df = query_df(daily_est_dsn, sql.format(device, date))
        db_df = db_device_df if len(db_df)==0 else pd.concat([db_df, db_device_df])
    db_df = db_df.drop_duplicates()
    print datetime.today()
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct().toPandas()
    print datetime.today()
    
    
    diff_df =db_df.merge(unified_df, indicator=True, how='left')
    diff_df = diff_df.loc[diff_df["_merge"] != "both"]
    
    print datetime.today()
    if len(diff_df) ==  0:
        print len(unified_df)
        print len(db_df)
        return "{};PASS".format(date)
    else:
        return "{};FAIL; {} {} {}".format(date, len(unified_df),len(db_df), diff_df.app_id.tolist()[0:5])
        
        
def write_log(strobj, s3obj):
    s3obj.put(Body=str(strobj))


In [0]:


s3 = boto3.resource('s3')
s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/all2/regression2.txt')
log = [] 

# 2010-07-04/   2020-02-21
date_list = get_date_list("2012-08-02", "2012-12-31")
date_list = ["2017-06-06"]
for date in date_list:
    temp_log = ""
    try:
        temp_log = compare(date)
        print temp_log
    except Exception, e:
        temp_log = "{}; ERROR; {}".format(date, e.message)
        print temp_log
        print e.message
    log.append(temp_log)
    write_log(log, s3object)